<a href="https://colab.research.google.com/github/balawillgetyou/dy/blob/master/BERTHuggingFaceTextClassifier20220909.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
!pip install transformers[sentencepiece]
#!pip install sentencepiece

In [4]:
import os
from fastai.imports import * #this imports important libraries like pandas and numpy, which can be directly imported as well

#this is a pyTorch implementation. Overly verbose comments are suppressed. DataLoader is an iterator to produce minibatches needed for training. But HF's DataCollatorWithPadding was used instead
import warnings,logging,torch
torch.cuda.empty_cache()
from torch.utils.data import DataLoader
warnings.simplefilter('ignore')
logging.disable(logging.WARNING)

#Hugging Face (HF) transformers
import transformers
from transformers import TrainingArguments,Trainer,AutoModelForSequenceClassification,AutoTokenizer,DataCollatorWithPadding

#Hugging Face utilities perform important functions, like casting the author labels from integers to categorical type using ClassLabel
import datasets
from datasets import load_dataset, Dataset, DatasetDict, ClassLabel

#for easy metrics generation
from sklearn.metrics import precision_recall_fscore_support as score

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
temp_1 = open("/content/gdrive/My Drive/Colab_Notebooks_Bala/WhoseLineIsItAnywayTRAIN.csv", 'r', encoding='latin-1') 
WhoseLineData = pd.read_csv(temp_1)
print(WhoseLineData.shape)
WhoseLineData.head(3)

(18977, 2)


,text,author
0,"They have been pronounced by an\n\n\n\n\nexpert to be of a rare variety and of considerable value., You can see\n\n\n\n\nfor yourselves that they are very handsome."" She opened a flat box as\n\n\n\n\nshe spoke, and showed me six of the finest pearls that I had ever seen.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n""Your statement is most interesting,"" said Sherlock Holmes. ""Has\n\n\n\n\nanything else occurred to you?""\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n""Yes, and no later than to-day., That is why I have come to you., This\n\n\n\n\nmorning I received this letter, which you will perhaps rea...",2
1,"His partner sailed along in\n\n\n\n\nfront of him, as though she noticed nothing., The medical student really did\n\n\n\n\ndance on his head, and excited frantic enthusiasm, stamping, and shrieks of\n\n\n\n\ndelight., In short, the absence of constraint was very marked., Ivan Ilyitch,\n\n\n\n\nwhom the wine was beginning to affect, began by smiling, but by degrees a\n\n\n\n\nbitter doubt began to steal into his heart; of course he liked free and\n\n\n\n\neasy manners and unconventionality., He desired, he had even inwardly prayed\n\n\n\n\nfor free and easy manners, when they had all held b...",0
2,"The cushions were a good deal higher\n\n\n\n than the balls, and as the balls had a fashion of always stopping under\n\n\n\n the cushions, we accomplished very little in the way of caroms., The\n\n\n\n cushions were hard and unelastic, and the cues were so crooked that in\n\n\n\n making a shot you had to allow for the curve or you would infallibly put\n\n\n\n the Ã¢â¬ÅEnglishÃ¢â¬ï¿½ on the wrong side of the ball., Dan was to mark while the\n\n\n\n doctor and I played., At the end of an hour neither of us had made a count,\n\n\n\n and so Dan was tired o...",5


In [7]:
WhoseLineData.author.value_counts()

0    3859
5    3517
4    3184
2    2621
3    1314
9    1142
7    1054
8     910
6     755
1     621
Name: author, dtype: int64

In [9]:
#trimming the dataset
df = pd.DataFrame(WhoseLineData.groupby('author').apply(lambda x: x.sample(500)))
print(df.shape)
df.author.value_counts()

(5000, 2)


0    500
1    500
2    500
3    500
4    500
5    500
6    500
7    500
8    500
9    500
Name: author, dtype: int64

In [10]:
ds = Dataset.from_pandas(df).rename_column('author', 'label') #flipping from Pandas dataframe to HF dataset
ds.features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 '__index_level_0__': Value(dtype='int64', id=None),
 '__index_level_1__': Value(dtype='int64', id=None)}

In [11]:
#model_nm = 'microsoft/deberta-v3-small' #using this pre-trained model caused out of memory errors
model_nm = 'distilbert-base-uncased'
tokz = AutoTokenizer.from_pretrained(model_nm, padding=True, truncation=True, max_length=512)#padding and truncation are needed to deal with the varying lengths 
#of inputs. max_length is to match the tensor length of the pre-trained model

def tok_func(x): return tokz(x["text"], truncation=True)
print(tok_func(ds[0]))
tok_ds = ds.map(tok_func, batched=True)
print(ds[0])
print(tok_ds[0])#scroll to the right to see. input_ids is just the specific word's sequence in the model's vocab. 
#'attention_mask' is set to 1 if attention is to be paid to the particular word, else 0

{'input_ids': [101, 2009, 2001, 1037, 2367, 3043, 2043, 2002, 2777, 2007, 29409, 2030, 2007, 2280, 3507, 2493, 1010, 3183, 1010, 5262, 1010, 2002, 18966, 3116, 2012, 2151, 2051, 1012, 1010, 1998, 2664, 2043, 1037, 15967, 2158, 2040, 1010, 2005, 2070, 4242, 3114, 1010, 2001, 2108, 2579, 4873, 1999, 1037, 4121, 11333, 13871, 2239, 7944, 2011, 1037, 3082, 2852, 4710, 3586, 1010, 3402, 6626, 2012, 2032, 2004, 2002, 5225, 2627, 1024, 1037, 29645, 2050, 29655, 4430, 3240, 2045, 1010, 2446, 6045, 14621, 29645, 2050, 29655, 2072, 1094, 1092, 8670, 13668, 2075, 2012, 1996, 2327, 1997, 2010, 2376, 1998, 7302, 2012, 2032, 2050, 29645, 2050, 29655, 8988, 2063, 2402, 2158, 3030, 3402, 1998, 13514, 29461, 12274, 15534, 2135, 2012, 2010, 6045, 1012, 1010, 2009, 2001, 1037, 4206, 2461, 6045, 2013, 27946, 2050, 29645, 2050, 29655, 2050, 29645, 2015, 1010, 2021, 3294, 6247, 2041, 1010, 13174, 2007, 2287, 1010, 2035, 7950, 1998, 2022, 13102, 20097, 2098, 1010, 7987, 5714, 3238, 1998, 6260, 2006, 2028, 22

  0%|          | 0/5 [00:00<?, ?ba/s]

{'text': 'It was a different matter when he met with acquaintances or\n\n\n\n\n      with former fellow students, whom, indeed, he disliked meeting at any\n\n\n\n\n      time., And yet when a drunken man who, for some unknown reason, was being\n\n\n\n\n      taken somewhere in a huge waggon dragged by a heavy dray horse, suddenly\n\n\n\n\n      shouted at him as he drove past: Ã¢â\x82¬Å\x93Hey there, German hatterÃ¢â\x82¬ï¿½ bawling at the\n\n\n\n\n      top of his voice and pointing at himÃ¢â\x82¬â\x80\x9dthe young man stopped suddenly\n\n\n\n\n      and clutched tremulously at his hat., It was a tall round hat from\n\n\n\n\n      ZimmermanÃ¢â\x82¬â\x84¢s, but completely worn out, rusty with age, all torn and\n\n\n\n\n      bespattered, brimless and bent on one side in a most unseemly fashion., Not\n\n\n\n\n      shame, however, but quite another feeling akin to terror had overtaken\n\n\n\n\n      him.\n\n\n\n\n    \n\n\n\n\n    \n\n\n\n\n      Ã¢â\x82¬Å\x93I knew it,Ã¢â\x82¬ï¿½ he mu

In [12]:
tok_ds = tok_ds.cast_column("label", ClassLabel(names=[0,1,2,3,4,5,6,7,8,9]))#column data type casting
train_test = tok_ds.train_test_split(test_size=0.1, stratify_by_column = 'label')#using HF's train_test split. Internally, this seems to be using sklearn's implementation :(

dds = DatasetDict({"train":train_test['train'],"test": train_test['test']})#converting to a HF specific data object/ dictionary

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
dds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__', '__index_level_1__', 'input_ids', 'attention_mask'],
        num_rows: 4500
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__', '__index_level_1__', 'input_ids', 'attention_mask'],
        num_rows: 500
    })
})

In [14]:
data_collator = DataCollatorWithPadding(tokenizer=tokz)#creates mini batches
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=10)#10 different authors and hence classes
lr,bs = 2e-5,16 #some hyper parameters used
wd,epochs = 0.01,4
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=lr,
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=2*bs,
    num_train_epochs=epochs,
    weight_decay=wd,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dds["train"],
    eval_dataset=dds["test"],
    tokenizer=tokz,
    data_collator=data_collator,
)

trainer.train()

Step,Training Loss
500,0.802300
1000,0.141600


TrainOutput(global_step=1128, training_loss=0.4265394481361335, metrics={'train_runtime': 892.1203, 'train_samples_per_second': 20.177, 'train_steps_per_second': 1.264, 'total_flos': 2384753356800000.0, 'train_loss': 0.4265394481361335, 'epoch': 4.0})

In [15]:
trainer.evaluate()

{'eval_loss': 0.14235332608222961,
 'eval_runtime': 8.7539,
 'eval_samples_per_second': 57.117,
 'eval_steps_per_second': 1.828,
 'epoch': 4.0}

In [16]:
preds = trainer.predict(dds["test"]).predictions.astype(float)#prediction on the test set itself. could be done differently with a separate validation set as well
preds = np.clip(preds, 0, 1)
predictions = np.argmax(preds, axis = 1)
targets = dds["test"]['label']

In [18]:
precision, recall, fscore, support = score(targets, predictions, labels = [0,1,2,3,4,5,6,7,8,9])
results1 = pd.concat((pd.Series([0,1,2,3,4,5,6,7,8,9]), pd.Series(precision), pd.Series(recall), pd.Series(fscore), pd.Series(support)), axis=1, join = 'outer')
results1.columns=('label','precision', 'recall', 'fscore', 'support')
print('Results','\n','*'*50,'\n',round(results1,2))

Results 
 ************************************************** 
    label  precision  recall  fscore  support
0      0       0.82    1.00    0.90       50
1      1       0.89    0.98    0.93       50
2      2       0.94    0.98    0.96       50
3      3       0.96    1.00    0.98       50
4      4       0.84    0.84    0.84       50
5      5       0.95    0.74    0.83       50
6      6       0.96    1.00    0.98       50
7      7       0.98    0.80    0.88       50
8      8       1.00    0.98    0.99       50
9      9       0.98    0.96    0.97       50
